```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Dependencies](#dependencies)**
2. **[Tables](#tables)**
3. **[Stored Procedures](#stored-procedures)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/DataFactory/Utils/Schemas.ipynb",
        "src/DataFactory/Utils/Protos.ipynb",
        "src/statics/municipalities/inhabitants_per_municipality.ipynb"
    ]
}
```

# **TABLES**

---

### STAGINGS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[RWZI_CONNECTED_INHABITANTS]'))
CREATE TABLE [VWSSTAGE].[RWZI_CONNECTED_INHABITANTS] (
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
	[REGIO_CODE] [VARCHAR](100) NULL,
	[REGIO_NAME] [VARCHAR](100) NULL,
    [CONNECTED_INHABITANTS] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

### STATICS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

IF NOT EXISTS(SELECT * FROM sys.sequences WHERE object_id = OBJECT_ID(N'[dbo].[SEQ_VWSSTATIC_RWZI_CONNECTED_INHABITANTS]') AND type = 'SO')
CREATE SEQUENCE SEQ_VWSSTATIC_RWZI_CONNECTED_INHABITANTS
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[RWZI_CONNECTED_INHABITANTS]'))
CREATE TABLE VWSSTATIC.RWZI_CONNECTED_INHABITANTS(
    [ID] INT PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [dbo].[SEQ_VWSSTATIC_RWZI_CONNECTED_INHABITANTS]),
	  [REGIO_CODE] [VARCHAR](100) NULL,
	  [REGIO_NAME] [VARCHAR](100) NULL,
    [CONNECTED_INHABITANTS] INT NULL,
	  [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

# **STORED PROCEDURES**

---

### STAGE &rarr; STATIC MAPPING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER PROCEDURE [DBO].[SP_RWZI_CONNECTED_INHABITANTS]
AS
BEGIN
    INSERT INTO [VWSSTATIC].[RWZI_CONNECTED_INHABITANTS] (        
        [REGIO_CODE],
	    [REGIO_NAME],
        [CONNECTED_INHABITANTS]
    )
    SELECT
        [REGIO_CODE],
	    [REGIO_NAME],
        [CONNECTED_INHABITANTS]
    FROM [VWSSTAGE].[RWZI_CONNECTED_INHABITANTS]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[RWZI_CONNECTED_INHABITANTS] WITH(NOLOCK))
END;
GO

### UPDATE INHABITANTS VIEW

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.


CREATE OR ALTER VIEW [VWSDEST].[V_INHABITANTS_PER_MUNICIPALITY]
AS
SELECT [GMCODE]
      ,[INHABITANTS] AS POPULATION_COUNT
      ,rwzi.[CONNECTED_INHABITANTS] AS POPULATION_COUNT_CONNECTED_TO_RWZIS
      ,mun.[DATE_LAST_INSERTED]
FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY] as mun
JOIN [VWSSTATIC].[RWZI_CONNECTED_INHABITANTS] as rwzi on rwzi.[REGIO_CODE] = mun.[GMCODE]
WHERE mun.DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM [VWSSTATIC].[INHABITANTS_PER_MUNICIPALITY])
AND rwzi.[DATE_LAST_INSERTED] = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM [VWSSTATIC].[RWZI_CONNECTED_INHABITANTS])

GO 

### UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS] adjust [COLUMNS] value for all GM PROTO CONFIGS

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport.
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

UPDATE [DATATINO_PROTO_1].[CONFIGURATIONS]
SET [COLUMNS] = 'POPULATION_COUNT|POPULATION_COUNT_CONNECTED_TO_RWZIS'
WHERE [NAME] = 'static_values' AND [COLUMNS] = 'POPULATION_COUNT'
AND PROTO_ID in (SELECT ID FROM  [DATATINO_PROTO_1].[PROTOS] WHERE NAME like 'GM%')